## Посимвольная языковая модель.

В первом задании Вам нужно написать и обучить посимвольную нейронную языковую модель для вычисления вероятностей буквенных последовательностей (то есть слов). Такие модели используются в задачах словоизменения и распознавания/порождения звучащей речи. Для обучения модели используйте данные для русского языка из [репозитория](https://github.com/sigmorphon/conll2018/tree/master/task1/surprise).

**В процессе написания Вам нужно решить следующие проблемы:**
    
* как будет выглядеть обучающая выборка; что будет являться признаками, и что - метками классов.
* как сделать так, чтобы модель при предсказании символа учитывала все предыдущие символы слова.
* какие специальные символы нужно использовать.
* как передавать в модель текущее состояние рекуррентной сети

**Результаты:**

* предобработчик данных,
* генератор обучающих данных (батчей),
* обученная модель
* перплексия модели на настроечной выборке
* посимвольные вероятности слов в контрольной выборке

**Дополнительно:**

* дополнительный вход модели (часть речи слова, другие морфологические признаки), влияет ли его добавление на перплексию
* сравнение различных архитектур нейронной сети (FC, RNN, LSTM, QRNN, ...)

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable

## Преобработка данных:

In [3]:
# # Uncomment to download data
# !wget https://github.com/sigmorphon/conll2018/blob/master/task1/surprise/russian-train-high
# !wget https://github.com/sigmorphon/conll2018/blob/master/task1/surprise/russian-dev
# !wget https://github.com/sigmorphon/conll2018/blob/master/task1/surprise/russian-covered-test

Переписала функцию считывания данных, потому что прошлая мне не нравилась.

In [3]:
def read_dataset(infile):
    df = pd.read_html(infile)
    df = df[0]
    df.columns = ["x", "y"]
    df["y"] = df["y"].astype('str')
    df = df.drop(columns=["x"])
    x = df.y.str.split("\t",expand=True,)
    if x.shape[1] == 3:
        df['word'] = x[0].str.lower()
        df['tag'] = x[2]
    else:
        df['word'] = x[0].str.lower()
        df['tag'] = x[1]
    df = df.drop(columns = ["y"])
    
    words = list(df['word'])
    tags = list(df['tag'])
    return words, tags

train_words, train_tags = read_dataset("russian-train-high")
dev_words, dev_tags = read_dataset("russian-dev")
test_words, test_tags = read_dataset("russian-covered-test")

Подумайте, какие вспомогательные токены могут быть вам полезны. Выдайте им индексы от `0` до `len(AUXILIARY) - 1`

Нам понадобится символ конца слова!

In [4]:
AUXILIARY = ['\n']

In [5]:
class Vocabulary:
    symbols = 0
    symbol_codes= dict()
    def fit(self, data):
        """Extract unique symbols from the data, make itos (item to string) and stoi (string to index) objects"""
        symbols = set(x for elem in data for x in elem)
        self.symbols = AUXILIARY + sorted(symbols)
        # Запомните следующую строчку кода - она нужна примерно всегда
        self.symbol_codes = {s: i for i, s in enumerate(self.symbols)}
        return self

    def __len__(self):
        return len(self._symbols)

    def transform(self, data):
        """Transform data to indices
        Input:
            - data, list of strings
        Output:
            - list of list of char indices

        >>> self.transform(['word1', 'token2'])
        >>> [[24, 2, 19, 13, 3], [8, 2, 9, 1, 7, 4]]
        """
        max_len = max(map(len, train_words))
        return [[self.symbol_codes[char] for char in word] + 
                [self.symbol_codes[AUXILIARY[0]]]*(max_len - len(word) + 1) for word in data]

In [6]:
v = Vocabulary()
v.fit(train_words)

In [7]:
transforned_train_words = v.transform(train_words)
transforned_dev_words = v.transform(dev_words)
transforned_test_words = v.transform(test_words)

In [8]:
max_len = max(map(len, transforned_train_words))
max_len

44

In [9]:
len(v.symbols)

36

## Генератор обучающих данных (батчей):

In [10]:
def batcher(data, batch_size = 4):
    data = np.array(data)
    data = data.reshape(int(data.shape[0]/batch_size), batch_size, data.shape[1])
    return data

## Модель:

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(len(v.symbols), input_size)
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        emb = self.embedding(input)
        input_combined = torch.cat((emb, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [19]:
criterion = nn.NLLLoss()

learning_rate = 1e-3

def train(input_line_tensor):
    
    hidden = rnn.initHidden()
    rnn.zero_grad()

    loss = 0

    n = len(input_line_tensor)
    for i in range(n - 1):
        input = torch.Tensor([input_line_tensor[i]]).long()
        target = torch.Tensor([input_line_tensor[i+1]]).long()
        
        output, hidden = rnn(input, hidden)
        l = criterion(output, target)
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / n

In [20]:
batch_size = 25

b_train = transforned_train_words# batcher(transforned_train_words, batch_size)
b_dev = transforned_dev_words #batcher(transforned_dev_words, batch_size)
b_test = transforned_test_words #batcher(transforned_test_words, batch_size)

rnn = RNN(10, 128, len(v.symbols))

n_batches = len(b_train)
print_every = n_batches/40
plot_every = n_batches/40
all_losses = []

for i in range(n_batches):
    output, loss = train(b_train[i])

    if i % print_every == 0:
        print('(%d %d%%) %.4f' % (i, i / n_batches * 100, loss))

    if i % plot_every == 0:
        all_losses.append(loss)

(0 0%) 3.4145
(250 2%) 0.7661
(500 5%) 0.7460
(750 7%) 0.6044
(1000 10%) 0.4997
(1250 12%) 0.6099
(1500 15%) 0.4826
(1750 17%) 0.8114
(2000 20%) 0.4596
(2250 22%) 0.7487
(2500 25%) 0.5445
(2750 27%) 0.7518
(3000 30%) 0.6156
(3250 32%) 0.4169
(3500 35%) 0.6279
(3750 37%) 0.3791
(4000 40%) 0.6267
(4250 42%) 0.6295
(4500 45%) 0.4878
(4750 47%) 0.5989
(5000 50%) 0.8679
(5250 52%) 0.6289
(5500 55%) 0.3339
(5750 57%) 0.8290
(6000 60%) 0.5071
(6250 62%) 0.5527
(6500 65%) 0.6039
(6750 67%) 0.7091
(7000 70%) 0.5788
(7250 72%) 0.6447
(7500 75%) 0.5301
(7750 77%) 0.9129
(8000 80%) 0.5520
(8250 82%) 0.4272
(8500 85%) 0.7750
(8750 87%) 0.5702
(9000 90%) 0.4974
(9250 92%) 0.6530
(9500 95%) 0.5785
(9750 97%) 0.6883


In [21]:
plt.figure(figsize = (10, 5))
plt.plot(all_losses)
plt.show()

NameError: name 'plt' is not defined

## Перплексия модели на настроечной выборке:

In [22]:
def test(input):
    with torch.no_grad():
        prob = []
        loss= 0
        predicted = [v.symbols[input[0]]]
        hidden = rnn.initHidden()

        for i in range(max_len-1):
            x =  torch.Tensor([input[i]]).long()
            output, hidden = rnn(x, hidden)
            target = input[i+1]
            l = criterion(output, torch.Tensor([target]).long())
            
            output = np.exp(list(output[0]))
            
            prob.append((v.symbols[target], output[target]))
            
            pred = np.argmax(output)
            letter = v.symbols[pred]
            if letter == '\n':
                if v.symbols[target] == '\n':
                    break
            else:
                predicted.append(letter)
                loss += l

        return predicted, prob, loss/len(predicted)

In [23]:
for num, i in enumerate(b_dev):
    total_loss = test(i)[2]

In [24]:
print("Перплексия модели на настроечной выборке:", float(2**(total_loss/len(b_dev))))

Перплексия модели на настроечной выборке: 1.0013777017593384


## Посимвольные вероятности слов в контрольной выборке:

In [28]:
i = 0
for batch in b_test[:10]:
    res = test(batch)[1]
    print(test_words[i])
    i += 1
    print(res)

мальтийский
[('а', 0.25050285), ('л', 0.054715242), ('ь', 0.1101205), ('т', 0.009931438), ('и', 0.086654566), ('й', 0.039271723), ('с', 0.009113473), ('к', 0.08444033), ('и', 0.20869167), ('й', 0.7848847), ('\n', 0.09413854), ('\n', 0.99466133)]
расчленить
[('а', 0.3061611), ('с', 0.10751787), ('ч', 0.0077340878), ('л', 0.0313337), ('е', 0.11856763), ('н', 0.21130224), ('и', 0.028132921), ('т', 0.18241128), ('ь', 0.8329007), ('\n', 0.5418521)]
лопаться
[('о', 0.2504094), ('п', 0.049267158), ('а', 0.10099998), ('т', 0.27762204), ('ь', 0.5657092), ('с', 0.17100863), ('я', 0.7871049), ('\n', 0.6988299)]
индексировать
[('н', 0.029501518), ('д', 0.019726587), ('е', 0.14537437), ('к', 0.03980444), ('с', 0.14421499), ('и', 0.080768764), ('р', 0.050032053), ('о', 0.09853223), ('в', 0.27753305), ('а', 0.19649972), ('т', 0.6828432), ('ь', 0.05573856), ('\n', 0.62151325)]
своевременный
[('в', 0.009151097), ('о', 0.14734623), ('е', 0.040518988), ('в', 0.016498104), ('р', 0.12405084), ('е', 0.12884

По вероятностям видно, что модель хорошо угадывает окончания.